In [2]:
# RNN(Long Short-Term Memory, LSTM)

In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# 讀取資料
def readTrain():
  train = pd.read_csv("S&P 500.csv")
  return train

In [4]:
# Augment Features
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

In [5]:
# Normalization
def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  print(np.mean(train))
  return train_norm

In [6]:
# Build Training Data
# X_train: 利用前30天的Open, High, Low, Close, Adj Close, Volume, month, year, date, day作為Features
# Y_train: 利用未來5天的Adj Close作為Features
# 須將資料做位移的展開作為Training Data
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

In [7]:
#資料亂序
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [8]:
# 將Training Data取一部份當作Validation Data
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [9]:
# 將輸出合併


# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)


Open         1.269477e+02
High         1.276978e+02
Low          1.261078e+02
Close        1.269418e+02
Adj Close    1.042098e+02
Volume       8.436351e+07
year         2.005170e+03
month        6.512606e+00
date         1.571841e+01
day          2.020328e+00
dtype: float64


In [11]:
# 模型建置

In [12]:
# 一對一模型
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [13]:
# 將過去的天數pastDay設為1，預測的天數futureDay也設為1
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 205us/step - loss: 0.0719 - val_loss: 0.0530
Epoch 2/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0423 - val_loss: 0.0288
Epoch 3/1000
5710/5710 [==============================] - 0s 14us/step - loss: 0.0202 - val_loss: 0.0113
Epoch 4/1000
5710/5710 [==============================] - 0s 8us/step - loss: 0.0065 - val_loss: 0.0027
Epoch 5/1000
5710/5710 [==============================] - 0s 11us/step - 

Epoch 68/1000
5710/5710 [==============================] - 0s 8us/step - loss: 3.7128e-05 - val_loss: 3.9514e-05
Epoch 69/1000
5710/5710 [==============================] - 0s 8us/step - loss: 3.6947e-05 - val_loss: 3.8459e-05
Epoch 70/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.6622e-05 - val_loss: 3.8001e-05
Epoch 71/1000
5710/5710 [==============================] - 0s 8us/step - loss: 3.6201e-05 - val_loss: 3.8839e-05
Epoch 72/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.6176e-05 - val_loss: 3.7540e-05
Epoch 73/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.5740e-05 - val_loss: 3.7264e-05
Epoch 74/1000
5710/5710 [==============================] - 0s 12us/step - loss: 3.5136e-05 - val_loss: 3.7081e-05
Epoch 75/1000
5710/5710 [==============================] - 0s 10us/step - loss: 3.4976e-05 - val_loss: 3.6376e-05
Epoch 76/1000
5710/5710 [==============================] - 0s 8us/step - loss: 3.5173e-05 -

Epoch 140/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4299e-05 - val_loss: 2.3772e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 11us/step - loss: 2.4421e-05 - val_loss: 2.3600e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4324e-05 - val_loss: 2.3893e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.3924e-05 - val_loss: 2.3471e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.3819e-05 - val_loss: 2.2929e-05
Epoch 145/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.3921e-05 - val_loss: 2.2980e-05
Epoch 146/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.3877e-05 - val_loss: 2.3387e-05
Epoch 147/1000
5710/5710 [==============================] - 0s 10us/step - loss: 2.4160e-05 - val_loss: 2.2622e-05
Epoch 148/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.4274

In [10]:
# 多對一模型
def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [11]:
# 需要設定的有pastDay=30、future=1 ，且Y_train 的維度需為二維
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
print(X_train.shape)
'''
model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])
'''

Open         1.269477e+02
High         1.276978e+02
Low          1.261078e+02
Close        1.269418e+02
Adj Close    1.042098e+02
Volume       8.436351e+07
year         2.005170e+03
month        6.512606e+00
date         1.571841e+01
day          2.020328e+00
dtype: float64
(5684, 30, 10)


'\nmodel = buildManyToOneModel(X_train.shape)\ncallback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")\nmodel.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])\n'

In [16]:
# 一對多模型
def buildOneToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
  model.add(Dense(1))
  model.add(RepeatVector(5))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [17]:
# 將pastDay 設為1, futureDay 設為5
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5706 samples, validate on 634 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 138us/step - loss: 0.0717 - val_loss: 0.0550
Epoch 2/1000
5706/5706 [==============================] - 0s 13us/step - loss: 0.0421 - val_loss: 0.0293
Epoch 3/1000
5706/5706 [==============================] - 0s 11us/step - loss: 0.0201 - val_loss: 0.0114
Epoch 4/1000
5706/5706 [=====================

5706/5706 [==============================] - 0s 12us/step - loss: 5.7311e-05 - val_loss: 5.4996e-05
Epoch 138/1000
5706/5706 [==============================] - 0s 10us/step - loss: 5.7560e-05 - val_loss: 5.7050e-05
Epoch 139/1000
5706/5706 [==============================] - 0s 9us/step - loss: 5.7824e-05 - val_loss: 5.5398e-05
Epoch 140/1000
5706/5706 [==============================] - 0s 13us/step - loss: 5.7564e-05 - val_loss: 5.5922e-05
Epoch 141/1000
5706/5706 [==============================] - 0s 10us/step - loss: 5.7269e-05 - val_loss: 5.6631e-05
Epoch 142/1000
5706/5706 [==============================] - 0s 8us/step - loss: 5.7574e-05 - val_loss: 5.5467e-05
Epoch 143/1000
5706/5706 [==============================] - 0s 12us/step - loss: 5.7079e-05 - val_loss: 5.6559e-05
Epoch 144/1000
5706/5706 [==============================] - 0s 8us/step - loss: 5.7266e-05 - val_loss: 5.5439e-05
Epoch 145/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7253e-05 - val_

In [18]:
# 多對多模型 (輸入與輸出相同長度)
def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [19]:
# 將pastDay 以及futureDay 設為相同長度5
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 10))`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5703 samples, validate on 633 samples
Epoch 1/1000
5703/5703 [==============================] - 1s 167us/step - loss: 0.0996 - val_loss: 0.0551
Epoch 2/1000
5703/5703 [==============================] - 0s 25us/step - loss: 0.0359 - val_loss: 0.0169
Epoch 3/1000
5703/5703 [==============================] - 0s 24us/step - loss: 0.0125 - val_loss: 0.0087
Epoch 4/1000
5703/5703 [==============================] - 0s 28us/step - loss: 0.0081 - val_loss: 0.0067
Epoch 5/1000
5703/5703 [==============================] - 0s 23us/step -

5703/5703 [==============================] - 0s 21us/step - loss: 8.8537e-05 - val_loss: 9.5499e-05
Epoch 140/1000
5703/5703 [==============================] - 0s 26us/step - loss: 8.8913e-05 - val_loss: 9.7076e-05
Epoch 141/1000
5703/5703 [==============================] - 0s 26us/step - loss: 8.7921e-05 - val_loss: 9.5884e-05
Epoch 142/1000
5703/5703 [==============================] - 0s 25us/step - loss: 8.8644e-05 - val_loss: 9.9321e-05
Epoch 143/1000
5703/5703 [==============================] - 0s 23us/step - loss: 8.7877e-05 - val_loss: 9.6408e-05
Epoch 144/1000
5703/5703 [==============================] - 0s 20us/step - loss: 8.9481e-05 - val_loss: 9.8832e-05
Epoch 145/1000
5703/5703 [==============================] - 0s 24us/step - loss: 8.8740e-05 - val_loss: 9.5387e-05
Epoch 146/1000
5703/5703 [==============================] - 0s 21us/step - loss: 8.8477e-05 - val_loss: 9.5474e-05
Epoch 147/1000
5703/5703 [==============================] - 0s 25us/step - loss: 8.9809e-05 - v